In [ ]:
# Kết nối Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Cài đặt các thư viện cần thiết
!pip install transformers datasets seqeval accelerate -U
!pip install vncorenlp

import torch
import os
from transformers import RobertaTokenizerFast, AutoModelForTokenClassification, TrainingArguments, Trainer
from datasets import Dataset, DatasetDict
from seqeval.metrics import classification_report, f1_score, precision_score, recall_score
import numpy as np

# Kiểm tra GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Đường dẫn tới file dữ liệu trên Google Drive
DRIVE_PATH = "/content/drive/MyDrive/"
SYLLABLE_DATA_FILE = os.path.join(DRIVE_PATH, "data_syllable.txt")
WORD_DATA_FILE = os.path.join(DRIVE_PATH, "data_word.txt")

# Kiểm tra file tồn tại
if not os.path.exists(SYLLABLE_DATA_FILE):
    raise FileNotFoundError(f"File không tồn tại: {SYLLABLE_DATA_FILE}")
if not os.path.exists(WORD_DATA_FILE):
    raise FileNotFoundError(f"File không tồn tại: {WORD_DATA_FILE}")

def read_conll_file(file_path):

    sentences = []
    current_tokens = []
    current_labels = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line == "":
                if current_tokens:
                    sentences.append({"tokens": current_tokens, "ner_tags_str": current_labels})
                    current_tokens = []
                    current_labels = []
            else:
                parts = line.split()
                if len(parts) >= 2:
                    token = " ".join(parts[:-1])
                    label = parts[-1]
                    current_tokens.append(token)
                    current_labels.append(label)
                else:
                    print(f"Dòng không hợp lệ, bỏ qua: '{line}' trong file {file_path}")
        if current_tokens:
            sentences.append({"tokens": current_tokens, "ner_tags_str": current_labels})
    return sentences

# Đọc dữ liệu
print("Đang đọc file âm tiết...")
syllable_sentences_data = read_conll_file(SYLLABLE_DATA_FILE)
print(f"Đã đọc {len(syllable_sentences_data)} câu từ file âm tiết.")

print("\nĐang đọc file từ...")
word_sentences_data = read_conll_file(WORD_DATA_FILE)
print(f"Đã đọc {len(word_sentences_data)} câu từ file từ.")

# xử lý và chia dữ liệu
if len(syllable_sentences_data) != len(word_sentences_data):
    print("CẢNH BÁO: Số lượng câu giữa file âm tiết và file từ không khớp!")
    min_len = min(len(syllable_sentences_data), len(word_sentences_data))
    syllable_sentences_data = syllable_sentences_data[:min_len]
    word_sentences_data = word_sentences_data[:min_len]
    print(f"Sử dụng {min_len} câu để đồng bộ.")
all_labels = sorted(list(set(label for sentence in syllable_sentences_data for label in sentence['ner_tags_str'])))
label_list = all_labels
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for i, label in enumerate(label_list)}
print("\nDanh sách nhãn (Label list):", label_list)
print("Số lượng nhãn:", len(label_list))
def convert_labels_to_ids(example):
    example['ner_tags'] = [label2id[label] for label in example['ner_tags_str']]
    return example
syllable_sentences_data = [convert_labels_to_ids(s) for s in syllable_sentences_data]
word_sentences_data = [convert_labels_to_ids(s) for s in word_sentences_data]
import random
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
num_sentences = len(syllable_sentences_data)
indices = list(range(num_sentences))
random.shuffle(indices)
train_split_idx = int(0.7 * num_sentences)
val_split_idx = int(0.85 * num_sentences)
train_indices = indices[:train_split_idx]
val_indices = indices[train_split_idx:val_split_idx]
test_indices = indices[val_split_idx:]
def get_subset(data, indices_subset): return [data[i] for i in indices_subset]
syllable_datasets = DatasetDict({
    "train": Dataset.from_list(get_subset(syllable_sentences_data, train_indices)),
    "validation": Dataset.from_list(get_subset(syllable_sentences_data, val_indices)),
    "test": Dataset.from_list(get_subset(syllable_sentences_data, test_indices))
})
word_datasets = DatasetDict({
    "train": Dataset.from_list(get_subset(word_sentences_data, train_indices)),
    "validation": Dataset.from_list(get_subset(word_sentences_data, val_indices)),
    "test": Dataset.from_list(get_subset(word_sentences_data, test_indices))
})
print("\nSyllable datasets:", syllable_datasets)
print("\nWord datasets:", word_datasets)


# Chuyển sang mô hình XLM-RoBERTa

MODEL_CHECKPOINT = "xlm-roberta-base"


# Tokenizer vẫn là RobertaTokenizerFast và cần add_prefix_space=True
tokenizer = RobertaTokenizerFast.from_pretrained(MODEL_CHECKPOINT, add_prefix_space=True)
print("Đã tải tokenizer cho:", MODEL_CHECKPOINT)
print("Loại tokenizer đã được tải:", type(tokenizer))

MAX_SEQ_LENGTH = 256
print(f"Sử dụng MAX_SEQ_LENGTH = {MAX_SEQ_LENGTH}")

def tokenize_and_align_labels(examples):

    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True,
        max_length=MAX_SEQ_LENGTH, padding="max_length"
    )
    labels = []
    for i, label_ids in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids_aligned = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids_aligned.append(-100)
            elif word_idx != previous_word_idx:
                label_ids_aligned.append(label_ids[word_idx])
            else:
                label_ids_aligned.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids_aligned)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

def train_and_evaluate(model_name, datasets_dict, data_type_name, output_dir_base):
    print(f"\n--- Bắt đầu xử lý cho mô hình {model_name} với dữ liệu: {data_type_name} ---")
    print("Tokenizing dữ liệu...")
    tokenized_datasets = datasets_dict.map(tokenize_and_align_labels, batched=True, num_proc=2)
    tokenized_datasets = tokenized_datasets.remove_columns(["tokens", "ner_tags_str", "ner_tags"])
    tokenized_datasets.set_format("torch")
    model = AutoModelForTokenClassification.from_pretrained(
        model_name, num_labels=len(label_list), id2label=id2label, label2id=label2id
    ).to(device)
    output_dir = os.path.join(output_dir_base, f"{model_name.replace('/', '-')}-kpi-ner-{data_type_name}")

    # tham số
    args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=10,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        weight_decay=0.01,
        logging_steps=100,
        save_steps=1000,
        report_to="none"
    )

    def compute_metrics(p):

        predictions, labels = p
        predictions = np.argmax(predictions, axis=2)
        true_predictions = [[id2label[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
        true_labels = [[id2label[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
        return {"precision": precision_score(true_labels, true_predictions, zero_division=0), "recall": recall_score(true_labels, true_predictions, zero_division=0), "f1": f1_score(true_labels, true_predictions, zero_division=0)}

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    print(f"Bắt đầu huấn luyện {model_name} cho dữ liệu {data_type_name}...")
    trainer.train()
    print("Hoàn tất huấn luyện.")

    print(f"\nĐánh giá trên tập test cho dữ liệu {data_type_name}...")
    test_results = trainer.evaluate(eval_dataset=tokenized_datasets["test"])
    print(f"Kết quả trên tập test ({data_type_name}):")
    for key, value in test_results.items():
        print(f"  {key}: {value:.4f}")

    final_model_path = os.path.join(output_dir, f"final_model_{data_type_name}")
    trainer.save_model(final_model_path)
    tokenizer.save_pretrained(final_model_path)
    print(f"Mô hình cuối cùng cho {data_type_name} đã được lưu tại: {final_model_path}")

    predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
    predictions = np.argmax(predictions, axis=2)
    true_predictions_test = [[id2label[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels_test = [[id2label[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    report_test_str = classification_report(true_labels_test, true_predictions_test, output_dict=False, digits=4, zero_division=0)
    print(f"\nBáo cáo chi tiết trên tập TEST cho {data_type_name}:\n{report_test_str}")
    return {"model": model_name, "data_type": data_type_name, "test_metrics": test_results, "classification_report_test": report_test_str, "best_model_path": final_model_path}

# Thư mục để lưu tất cả các output
ALL_OUTPUTS_DIR = os.path.join(DRIVE_PATH, "kpi_ner_outputs_xlmr") # Tạo thư mục mới cho XLM-R
os.makedirs(ALL_OUTPUTS_DIR, exist_ok=True)
results_xlmr = []

# Chạy thí nghiệm với dữ liệu âm tiết
result_syllable = train_and_evaluate(MODEL_CHECKPOINT, syllable_datasets, "syllable", ALL_OUTPUTS_DIR)
results_xlmr.append(result_syllable)

# Chạy thí nghiệm với dữ liệu từ
result_word = train_and_evaluate(MODEL_CHECKPOINT, word_datasets, "word", ALL_OUTPUTS_DIR)
results_xlmr.append(result_word)

print("\n\n--- TỔNG HỢP KẾT QUẢ XLM-ROBERTA ---")
for res in results_xlmr:
    print(f"\nKết quả cho mô hình {res['model']} với dữ liệu {res['data_type']}:")
    print("  Metrics trên tập Test:")
    for k, v in res['test_metrics'].items():
        if k.startswith("eval_"):
             print(f"    {k}: {v:.4f}")
    print(f"  Đường dẫn mô hình: {res['best_model_path']}")
    print("  Báo cáo chi tiết trên tập Test:")
    print(res['classification_report_test'])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using device: cuda
Đang đọc file âm tiết...
Đã đọc 2500 câu từ file âm tiết.

Đang đọc file từ...
Đã đọc 2500 câu từ file từ.

Danh sách nhãn (Label list): ['B-DIRECTION', 'B-KPI', 'B-OWNER', 'B-TARGET', 'B-TIME_FRAME', 'B-UNIT', 'I-DIRECTION', 'I-KPI', 'I-OWNER', 'I-TARGET', 'I-TIME_FRAME', 'I-UNIT', 'O']
Số lượng nhãn: 13

Syllable datasets: DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags_str', 'ner_tags'],
        num_rows: 1750
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags_str', 'ner_tags'],
        num_rows: 375
    })
    test: Dataset({
        features: ['tokens', 'ner_tags_str', 'ner_tags'],
        num_rows: 375
    })
})

Word datasets: DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags_str', 'ner_tags'],
        num_rows: 1750
    })
    validation: Dataset({
        featur

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

...

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-3-505280339d9a>:180: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Bắt đầu huấn luyện xlm-roberta-base cho dữ liệu word...


Step,Training Loss
100,0.515800
200,0.104400
300,0.073500
400,0.054100
500,0.044300
600,0.036000
700,0.051200
800,0.022800
900,0.021800
1000,0.025700


Hoàn tất huấn luyện.

Đánh giá trên tập test cho dữ liệu word...


Kết quả trên tập test (word):
  eval_loss: 0.0230
  eval_precision: 0.9907
  eval_recall: 0.9898
  eval_f1: 0.9902
  eval_runtime: 5.2367
  eval_samples_per_second: 71.6100
  eval_steps_per_second: 8.9750
  epoch: 10.0000
Mô hình cuối cùng cho word đã được lưu tại: /content/drive/MyDrive/kpi_ner_outputs_xlmr/xlm-roberta-base-kpi-ner-word/final_model_word

Báo cáo chi tiết trên tập TEST cho word:
              precision    recall  f1-score   support

   DIRECTION     0.9796    0.9796    0.9796       343
         KPI     0.9947    0.9947    0.9947       375
       OWNER     1.0000    1.0000    1.0000       372
      TARGET     0.9892    0.9839    0.9865       372
  TIME_FRAME     0.9915    0.9886    0.9900       352
        UNIT     0.9882    0.9911    0.9897       338

   micro avg     0.9907    0.9898    0.9902      2152
   macro avg     0.9905    0.9896    0.9901      2152
weighted avg     0.9907    0.9898    0.9902      2152



--- TỔNG HỢP KẾT QUẢ XLM-ROBERTA ---

Kết quả cho mô hìn